In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn import preprocessing
from matplotlib.backends.backend_pdf import PdfPages
from typing import List, Dict

warnings.filterwarnings("ignore")
plt.rcParams["font.size"] = 10

In [3]:
######### Load each data frame into dictionary - Permian ##########
file_names = ! ls /home/savannah/quantico/Permian/
dict_dfs = {}
for file_name in file_names:
    df=pd.read_csv(f'/home/savannah/quantico/Permian/{file_name}')
    dict_dfs[file_name]=df
    
# these are the cols of interest
consistent_col_names =['DEPTH(ft)',
 'ROP(ft / min)',
 'WOB(klbs)',
 'TPO(gal / min)',
 'RPM(rpm)',
 'SPP(psi)',
 'INC(deg)',
 'Gamma(gapi)',
 'DIFP(psi)',
 'BIT(in)']

In [65]:
#### create spike filters ####
def remove_outliers(feature:np.ndarray[np.float64],col:str,outlier_vars:Dict[str, float])->np.ndarray[np.float64]:
    """Remove outliers beyond x*std in drill feature data.

    Args:
        feature (np.ndarray[np.float64]): drill feature data
        col (str): name of drill feature
        outlier_vars (Dict[str, float]): dictionary with keys of features we want to remove outliers and value is x in (x*std formula)

    Returns:
        np.ndarray[np.float64]: feature data with outliers replaced with mean of data
    """    
    # filter outliers
    x=outlier_vars[col]
    # outliers = (feature>=(feature.mean() + feature.std()*x)) | (feature<=(feature.mean() - feature.std()*x))
    # feature = np.where(outliers,feature.mean(),feature)
    end_indx = 150
    new_features=[]
    for i in range(0, len(feature)):
        # get the mean of the surrounding end_indx*2 feature
        window=np.concatenate([feature[ i - end_indx : i], feature[i : i+ end_indx+1]])
        med_surround_feature = np.mean(window)
        if ((feature[i]>=(feature.mean() + feature.std()*x)) | (feature[i]<=(feature.mean() - feature.std()*x))):
            new_features.append(med_surround_feature)
        
        else:
            new_features.append(feature[i])
    return np.array(new_features)

#### plot_spike v despike data ####
def plot_relationships(df:pd.DataFrame,consistent_col_names:List)->plt.Figure:
    """Plot despiked vs not despiked data for each feature of interest and each dataset.

    Args:
        df (pd.DataFrame): dataset of features with corresponding Depth.
        consistent_col_names (List): list of column names of features of interest plus depth.

    Returns:
        plt.Figure: scatter plot of depth v spiked/despiked feature data.
    """    
    cols=consistent_col_names[1:]
    fig,ax=plt.subplots(1,len(cols),figsize=(20,10),constrained_layout=True)
    outlier_vars={'ROP(ft / min)':15.0, 'WOB(klbs)':7.0, 'SPP(psi)':4, 'Gamma(gapi)':5, 'DIFP(psi)':10.0,'TPO(gal / min)':20}
    dfs=pd.DataFrame(df['DEPTH(ft)'])
    for i,col in enumerate(cols):
        if (col != 'Gamma(gapi)') & (col != 'INC(deg)'):
            df1 = df[['DEPTH(ft)','Gamma(gapi)',col,'INC(deg)']].dropna() #TODO missing data strategy here
        else:
            df1 = df[['DEPTH(ft)','Gamma(gapi)','INC(deg)']].dropna() #TODO missing data strategy here
            
        sns.lineplot(data=df1,y='DEPTH(ft)',x=col,color='#e28743',alpha=1,label='Original',orient="y",ax=ax[i])
        if col not in ['INC(deg)','RPM(rpm)','BIT(in)']:
            x=outlier_vars[col]
            horz = remove_outliers(np.array(df1[df1['INC(deg)']>80][col]),col,outlier_vars)
            vert = remove_outliers(np.array(df1[df1['INC(deg)']<=80][col]),col,outlier_vars)
            df1[f'{col}_despiked'] = np.concatenate([vert,horz])
            sns.lineplot(data=df1,y='DEPTH(ft)',x=f'{col}_despiked',color='#154c79',alpha=1,label=f'Despiked (x={x})',orient="y",ax=ax[i])

        ax[0].set_ylabel('DEPTH(ft)',fontsize=16,labelpad=10)
        if i !=0:
            ax[i].set_ylabel('')
            ax[i].set_yticklabels('')
        ax[i].set_xlabel('')

        leg=ax[i].legend(frameon=False,loc='upper center',bbox_to_anchor=(.5,1.18),fontsize=12,)
        leg._legend_box.align = "bottom"

        ax[i].set_ylim(df1['DEPTH(ft)'].min()-100,df1['DEPTH(ft)'].max()+100)
        ax[i].set_title(col,fontsize=14,color='#873e23')
        ax[i].invert_yaxis()
        
        dfs=dfs.merge(df1[['DEPTH(ft)',col]],on='DEPTH(ft)',how='outer')
        if df1['DEPTH(ft)'].diff().value_counts().shape[0]>1:
            print('missing data in',col)

    fig.suptitle(f"Despiked data - x*std outliers within surrounding 50ft window corrected with median of window\n{file_name.replace('.csv','')}",horizontalalignment='left',fontsize=18,x=0.05,y=.95,)
    fig.tight_layout(rect=[0, 0, 1, 0.95])
    
    plt.close()
    return fig, dfs



In [66]:

pdf = PdfPages("figs/despiked_permiandata_120523.pdf")
figs=[]
for file_name in file_names:
    df=dict_dfs[file_name][consistent_col_names]
    fig, df1 = plot_relationships(df,consistent_col_names)
    df1.to_csv(f'permian_despiked_120523/{file_name}')
    figs.append(fig)
    plt.close()
for fig in figs:
    pdf.savefig(fig) # saves the active handle
pdf.close()


missing data in BIT(in)
